<a href="https://colab.research.google.com/github/danilobml/pytorch-networks/blob/main/basic_image_classifying_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Get datasets:
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.MNIST(root='.', train=True, download=True, transform=ToTensor())
test_data = datasets.MNIST(root='.', train=False, download=True, transform=ToTensor())

In [ ]:
# Load data with DataLoaders
from torch.utils.data import DataLoader

loaded_train = DataLoader(training_data, batch_size=64, shuffle=True)
loaded_test = DataLoader(test_data, batch_size=64, shuffle=True)

In [ ]:
# Create the Neural Network model:
from torch import nn

class NeuralNetwork(nn.Module):
  def __init__(self):
    # Call super, with this class as argument:
    super(NeuralNetwork, self).__init__()
    # Transform the data from multidimensional to one dimension only:
    self.flatten = nn.Flatten()
    # Create a sequence of layers to transform the data inside the network:
    self.linear_relu_stack = nn.Sequential(
        # Applies an affine linear transformation to the incoming data:
        # (size_of_input, size_of_output)
        nn.Linear(28*28, 512),
        # Applies the rectified linear unit function element-wise:
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )

  # forward is called when model is executed:
  def forward(self, x):
      x = self.flatten(x)
      logits = self.linear_relu_stack(x)
      return logits


# instantiate the model:
model = NeuralNetwork()
print(model)

In [ ]:
import torch
# Training the model:
# 1. set a loss function -> measures how far our model is from the correct
# results, and it's what we'll try to minimize during the training of the network.
# Cross-entropy is a common loss function used for classification tasks

loss_function = nn.CrossEntropyLoss()

# 2. set optimization algorythm -> adjusts the model during the training process
# in order to minimize the error measured by the loss function. A common choice
# for this kind of task is the stochastic gradient descent algorithm (SGD).
# (model params, learning rate: float ->  speed at which the model's parameters
# will be updated during each iteration in training)

optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [ ]:
# Train and test the network:

# For each of these tasks, we'll implement a function.

# 1. Train function -> consists of looping through the data one batch at a time,
# using the optimizer to adjust the model, and computing the prediction and the
# loss:

def train(dataloader: DataLoader, model: NeuralNetwork, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    pred = model(X)
    loss = loss_fn(pred, y)

    # Resets the gradients of all optimized torch.Tensor:
    optimizer.zero_grad()
    loss.backward()
    # Performs a single optimization step:
    optimizer.step()

    if batch:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
# 2. Test function -> computes the accuracy and the loss, this time using the
# test set:

def test(dataloader: DataLoader, model: NeuralNetwork, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0
  # Context-manager that disables gradient calculation:
  with torch.no_grad():
    for X, y in dataloader:
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
# 3. Set the number of epochs to train our model.
# An epoch consists of an iteration over the dataset:

epochs = 7

# RUN!!
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train(loaded_train, model, loss_function, optimizer)
  test(loaded_test, model, loss_function)

print("Done!")

In [ ]:
# With the model finally trained, save it and load it when necessary:

torch.save(model, 'model.pth')
model1 = torch.load('model.pth')

In [ ]:
# Use the model to predict the class of an image:
from PIL import Image
from torchvision.transforms import ToTensor, Compose, Normalize

# import image from google drive:
from google.colab import drive
drive.mount('/content/drive')

# 1.Load the image
image = Image.open('number_image.png').convert('L')
image.save('/content/drive/My Drive/number_image.png')

# 2. Resize the image
# image = image.resize((28, 28)) # Resize the image to 28x28 pixels

# 3.Preprocess the image
transform = Compose([
    ToTensor(),
    Normalize((0.1307,), (0.3081,)) # Normalize the image
])
image_tensor = transform(image)

# 4.Make a prediction
with torch.no_grad():
  output = model1(image_tensor)
  predicted_class = output.argmax(1).item()

print(f'Predicted class: {predicted_class}')